In [2]:
# %%

# import math # roundは組み込み関数のため不要
# jax関連のライブラリを全てインポート
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import vmap

# from LikelihoodEvaluator import LikelihoodEvaluator
# from ParameterEstimator import ParameterEstimator
from sympy import Array, symbols

from degenerate_sim import (
    DegenerateDiffusionProcess,
    LikelihoodEvaluator,
    m_estimate,
    newton_solve,
    m_estimate_jax,
    bayes_estimate
)
from degenerate_sim.evaluation.likelihood_evaluator_jax import SymbolicPreparationError

x_sym, y_sym = symbols("x, y")
theta_10, theta_20, theta_21 = symbols("theta_10 theta_20 theta_21")
theta_30, theta_31 = symbols("theta_30 theta_31")

# ベクトル化（1次元テンソルとして）
x = Array([x_sym])
y = Array([y_sym])
theta_1 = Array([theta_10])
theta_2 = Array([theta_20, theta_21])
theta_3 = Array([theta_30])

# Step 2: A, B, H の式定義
A = Array([-theta_20*x_sym  - theta_21*y_sym])
B = Array([[theta_10]])
H = Array([(theta_30 * x_sym)])

# インスタンス
FNmodel = DegenerateDiffusionProcess(
    x=x,
    y=y,
    theta_1=theta_1,
    theta_2=theta_2,
    theta_3=theta_3,
    A=A,
    B=B,
    H=H,
)
FN_likelihood = LikelihoodEvaluator(FNmodel)

true_theta1 = jnp.array([1.0])
true_theta2 = jnp.array([1.0, 1.0])
true_theta3 = jnp.array([1.0])
true_theta = (true_theta1, true_theta2, true_theta3)
t_max = 10
burn_out = 50
h = 0.05
bounds_theta1 = [(0.1, 2.0)]
bounds_theta2 = [(0.1, 2.0), (0.1, 2.0)]
bounds_theta3 = [(0.1, 2.0)]
n = t_max//h
x_series, y_series = FNmodel.simulate(true_theta=true_theta, t_max=t_max, burn_out=burn_out, h=h)


In [4]:
import numpy as np
from degenerate_sim.estimation.loop_estimation_algorithm import (
    LoopEstimationAlgorithm,
    SimulationConfig,
)

loop_plan = {
    1: ("B", "B", "B"),
    2: ("M", "M", "M"),
    3: ("M", "M", "M")
    
}

loop_sim_cfg = SimulationConfig(
    true_theta=true_theta,
    t_max=100.0,
    h=0.2,
    burn_out=50.0,
    dt=0.0001,
)

loop_estimator = LoopEstimationAlgorithm(
    evaluator=FN_likelihood,
    simulation_config=loop_sim_cfg,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
)

# loop_results = loop_estimator.run(
#     seeds=range(10),
#     plan=loop_plan,
#     max_iteration = 1,
#     initial_theta_stage0=(
#         jnp.array([0.2]),
#         jnp.array([0.5, 0.5]),
#         jnp.array([0.2, 0.1]),
#     ),
# )

# for seed, seq in loop_results.items():
#     print(f"Seed {seed}:")
#     for est in seq:
#         theta1_vals = ", ".join(f"{float(val):.6f}" for val in np.asarray(est.theta_final[0]))
#         theta2_vals = ", ".join(f"{float(val):.6f}" for val in np.asarray(est.theta_final[1]))
#         theta3_vals = ", ".join(f"{float(val):.6f}" for val in np.asarray(est.theta_final[2]))
#         print(
#             f"  iter {est.k}: theta1=[{theta1_vals}], "
#             f"theta2=[{theta2_vals}], theta3=[{theta3_vals}]"
#         )
#     print()


In [10]:
import pandas as pd

In [15]:
theta_list_1 = []
loop_estimator = LoopEstimationAlgorithm(
    evaluator=FN_likelihood,
    simulation_config=loop_sim_cfg,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
)
for seed in range(100):
    observation = loop_estimator._simulate_observation(seed = seed)
    theta = loop_estimator._estimate_component(
        estimator_kind ="M",
        component=1,
        k_arg=1,
        observation= observation,
        theta_bar =  true_theta,
        initial_guess= [0.3],

    )
    theta_list_1.append(theta)
print(pd.DataFrame(theta_list_1).describe())


                0
count  100.000000
mean     1.004985
std      0.017508
min      0.959260
25%      0.993244
50%      1.003266
75%      1.017387
max      1.047089


In [14]:
theta_list_2 = []
loop_estimator = LoopEstimationAlgorithm(
    evaluator=FN_likelihood,
    simulation_config=loop_sim_cfg,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
)
for seed in range(100):
    observation = loop_estimator._simulate_observation(seed = seed)
    theta = loop_estimator._estimate_component(
        estimator_kind ="M",
        component=1,
        k_arg=1,
        observation= observation,
        theta_bar =  true_theta,
        initial_guess= [0.3],
        my_setting= False

    )
    theta_list_1.append(theta)
print(pd.DataFrame(theta_list_1).describe())


                0
count  120.000000
mean     0.958444
std      0.023702
min      0.905638
25%      0.943131
50%      0.953817
75%      0.971568
max      1.039760


In [17]:
loop_estimator = LoopEstimationAlgorithm(
    evaluator=FN_likelihood,
    simulation_config=loop_sim_cfg,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
)
for seed in range(10):
    observation = loop_estimator._simulate_observation(seed = seed)
    print(loop_estimator._estimate_component(
        estimator_kind ="M",
        component=1,
        k_arg=2,
        observation= observation,
        theta_bar =  true_theta,
        initial_guess= [0.3],
        my_setting= False
    ))

[0.32704663]
[0.3471516]
[0.35869607]
[0.33564186]
[0.3155959]
[0.33958328]
[0.35539973]
[0.3273841]
[0.3213455]
[0.34118876]


In [19]:
loop_estimator = LoopEstimationAlgorithm(
    evaluator=FN_likelihood,
    simulation_config=loop_sim_cfg,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
)
for seed in range(10):
    observation = loop_estimator._simulate_observation(seed = seed)
    print(loop_estimator._estimate_component(
        estimator_kind ="M",
        component=1,
        k_arg=3,
        observation= observation,
        theta_bar =  true_theta,
        initial_guess= [0.3],
        my_setting= False
    ))

[0.2929161]
[0.31287515]
[0.2730288]
[0.28879026]
[0.29765156]
[0.30495548]
[0.2977807]
[0.27666748]
[0.29143763]
[0.2856269]


In [3]:
import numpy as np
from degenerate_sim.estimation.loop_estimation_algorithm import (
    LoopEstimationAlgorithm,
    SimulationConfig,
)

loop_plan = {
    1: ("B", "B", "B"),
    2: ("M", "M", "M"),
    3: ("M", "M", "M")
    
}

loop_sim_cfg = SimulationConfig(
    true_theta=true_theta,
    t_max=100.0,
    h=0.05,
    burn_out=50.0,
    dt=0.0001,
)

loop_estimator = LoopEstimationAlgorithm(
    evaluator=FN_likelihood,
    simulation_config=loop_sim_cfg,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
)

loop_results = loop_estimator.run(
    seeds=range(10),
    plan=loop_plan,
    max_iteration = 1,
    initial_theta_stage0=(
        jnp.array([0.2]),
        jnp.array([0.5, 0.5]),
        jnp.array([0.2, 0.1]),
    ),
)

for seed, seq in loop_results.items():
    print(f"Seed {seed}:")
    for est in seq:
        theta1_vals = ", ".join(f"{float(val):.6f}" for val in np.asarray(est.theta_final[0]))
        theta2_vals = ", ".join(f"{float(val):.6f}" for val in np.asarray(est.theta_final[1]))
        theta3_vals = ", ".join(f"{float(val):.6f}" for val in np.asarray(est.theta_final[2]))
        print(
            f"  iter {est.k}: theta1=[{theta1_vals}], "
            f"theta2=[{theta2_vals}], theta3=[{theta3_vals}]"
        )
    print()



Seed 0:
  iter 1: theta1=[0.378079], theta2=[1.597582, 0.878172], theta3=[0.105985, 0.000110]

Seed 1:
  iter 1: theta1=[0.402558], theta2=[1.473071, 0.769509], theta3=[0.105365, -0.000621]

Seed 2:
  iter 1: theta1=[0.400224], theta2=[1.461332, 0.763970], theta3=[0.105697, -0.000526]

Seed 3:
  iter 1: theta1=[0.417582], theta2=[1.497630, 0.767952], theta3=[0.105921, -0.000054]

Seed 4:
  iter 1: theta1=[0.392520], theta2=[1.421985, 0.765921], theta3=[0.106551, -0.000394]

Seed 5:
  iter 1: theta1=[0.380315], theta2=[1.401363, 0.736407], theta3=[0.105941, 0.000006]

Seed 6:
  iter 1: theta1=[0.397475], theta2=[1.467673, 0.833476], theta3=[0.106262, -0.000157]

Seed 7:
  iter 1: theta1=[0.383378], theta2=[1.412952, 0.785421], theta3=[0.106665, -0.000028]

Seed 8:
  iter 1: theta1=[0.364602], theta2=[1.376674, 0.777070], theta3=[0.105874, 0.000170]

Seed 9:
  iter 1: theta1=[0.393633], theta2=[1.446466, 0.754858], theta3=[0.105444, -0.001109]

